# Teste no AMLSim (versão Google Colab)

## Clonando repositório Github

In [1]:
!git clone https://github.com/italodellagarza/GCNMoneyLaundering

Cloning into 'GCNMoneyLaundering'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 54 (delta 20), reused 49 (delta 15), pack-reused 0
Unpacking objects: 100% (54/54), done.


## Entrando do diretório correto de trabalho

In [2]:
%cd GCNMoneyLaundering/test_amlsim_nenn/

/content/GCNMoneyLaundering/test_amlsim_nenn


## Verificando configurações da placa de vídeo

In [3]:
!nvidia-smi

Sun Mar 27 16:59:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    24W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Verificando versão do Pytorch

In [4]:
import torch; print(torch.__version__)

1.10.0+cu111


## Instalando Pytorch Geometric

In [5]:
pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.10.0+cu111.html

Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 7.9 MB 5.3 MB/s 
     |████████████████████████████████| 3.5 MB 23.1 MB/s 
     |████████████████████████████████| 2.5 MB 27.3 MB/s 
     |████████████████████████████████| 750 kB 36.5 MB/s 
     |████████████████████████████████| 407 kB 5.1 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=9978b58bd2f1dd008a02f0aee6d933e7dffc1d9e4096be056a6132f39734cc01
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric


## Importando Bibliotecas

In [6]:
import os
from torch_geometric.data import Data
import torch

## Montando Diretório do Google Drive

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Importando Dataset já convertido do Google Drive

In [8]:
dataset = []

for ptfile in os.listdir('/content/gdrive/MyDrive/Materiais Dissertação/Datasets/10Kvertices-1Medges-oversampling'):
    dataset.append(torch.load(f'/content/gdrive/MyDrive/Materiais Dissertação/Datasets/10Kvertices-1Medges-oversampling/{ptfile}'))

## Divisão treino e teste (50% cada)

In [9]:
train_data = dataset[0:int(0.8*200)]
test_data = dataset[int(0.8*200):]

## Algumas verificações da base de dados

In [10]:
dataset[1].edge_to_edge_adj_matr.count_nonzero()

tensor(19)

In [11]:
dataset[1].y

tensor([0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1])

In [12]:
dataset[1].edge_attr

tensor([[1.7136e-07, 9.8487e-07, 4.8564e-06, 2.2352e-08, 7.9721e-07, 5.5879e-08,
         6.5193e-08, 2.4752e-05, 5.8948e-06, 9.3132e-09, 5.8832e-06, 6.8275e-06,
         7.9544e-06, 8.5682e-08, 6.1234e-07, 3.6554e-07, 4.8382e-07, 4.1537e-07,
         6.6403e-07],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 2.9000e-01, 0.0000e+00, 4.6000e-01,
         1.5500e-01, 0.0000e+00, 0.0000e+00, 4.7500e-01, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 8.2500e-02, 0.0000e+00, 8.6750e-01, 5.0750e-01, 0.0000e+00,
         6.7000e-01]], dtype=torch.float64)

In [13]:
len(train_data)

160

In [14]:
len(test_data)

40

In [16]:
dataset[0].x.shape[0]

2

In [17]:
dataset[0].edge_attr.shape[0]

2

## Importando o Modelo

In [15]:
from model_nenn_simplificado import Nenn

## Criando uma instância do modelo

In [25]:
model = Nenn(2,2,2,2,2)

Transferindo o modelo para a CUDA

In [26]:
model = model.cuda()

## Definindo os parâmetros de treinamento

In [27]:
loss = torch.nn.CrossEntropyLoss().cuda()
# TODO Undersampling Smote
# TODO Oversampling
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

## Treino

In [29]:
def train():
    model.train()
    # Enumerar sobre os dados.
    for ts, data in enumerate(train_data):
        for epoch in range(1000):
            # Resetar Gradientes
            optimizer.zero_grad()
            # Passando as informações do batch e de conexão dos grafos
            logits = model(
                data.x.T.type(torch.FloatTensor).cuda(),
                data.edge_attr.T.type(torch.FloatTensor).cuda(),
                data.edge_to_edge_adj_matr.type(torch.FloatTensor).cuda(),
                data.edge_to_node_adj_matr.type(torch.FloatTensor).cuda(),
                data.node_to_edge_adj_matr.type(torch.FloatTensor).cuda(),
                data.node_to_node_adj_matr.type(torch.FloatTensor).cuda()
            )
            #label_pred = pred.max(1)[1]
            # Calculando a perda e os gradientes.
            l = loss(logits.cuda(), data.y.cuda())
            l.backward()
            # Atualizar usando os gradientes.
            optimizer.step()
            if (epoch+1) % 100 == 0:
                print('ts',ts+1,'epoch =', epoch + 1, 'loss =', l.item())
            torch.cuda.empty_cache() 

In [30]:
torch.cuda.empty_cache() 

In [31]:
train()

ts 1 epoch = 100 loss = 0.7225459218025208
ts 1 epoch = 200 loss = 0.7071132063865662
ts 1 epoch = 300 loss = 0.694414496421814
ts 1 epoch = 400 loss = 0.684389591217041
ts 1 epoch = 500 loss = 0.6767159104347229
ts 1 epoch = 600 loss = 0.6708765625953674
ts 1 epoch = 700 loss = 0.6663920283317566
ts 1 epoch = 800 loss = 0.6626111268997192
ts 1 epoch = 900 loss = 0.6593422889709473
ts 1 epoch = 1000 loss = 0.6565468311309814
ts 2 epoch = 100 loss = 0.7182660102844238
ts 2 epoch = 200 loss = 0.7022254467010498
ts 2 epoch = 300 loss = 0.6906344294548035
ts 2 epoch = 400 loss = 0.682233989238739
ts 2 epoch = 500 loss = 0.6758811473846436
ts 2 epoch = 600 loss = 0.6707202792167664
ts 2 epoch = 700 loss = 0.6661654710769653
ts 2 epoch = 800 loss = 0.6618303060531616
ts 2 epoch = 900 loss = 0.6574655771255493
ts 2 epoch = 1000 loss = 0.6529170274734497
ts 3 epoch = 100 loss = 0.660072922706604
ts 3 epoch = 200 loss = 0.6510841250419617
ts 3 epoch = 300 loss = 0.6441603899002075
ts 3 epoch = 

## Teste

In [32]:
label_pred_list = []
y_true_list = []

def test():
    model.eval()
    with torch.no_grad():
        global label_pred_list
        global y_true_list
        
        for data in test_data:
            data.to('cuda')
            print(data.node_to_edge_adj_matr.shape)
            logits = model(
                data.x.T.type(torch.FloatTensor).cuda(),
                data.edge_attr.T.type(torch.FloatTensor).cuda(),
                data.edge_to_edge_adj_matr.type(torch.FloatTensor).cuda(),
                data.edge_to_node_adj_matr.type(torch.FloatTensor).cuda(),
                data.node_to_edge_adj_matr.type(torch.FloatTensor).cuda(),
                data.node_to_node_adj_matr.type(torch.FloatTensor).cuda()
            )
            label_pred = logits.max(1)[1].tolist()
            label_pred_list += label_pred
            y_true_list += data.y.tolist()
    model.train()

In [33]:
torch.diag_embed(torch.rand([481, 443])).shape

torch.Size([481, 443, 443])

In [34]:
test()

torch.Size([30, 16])
torch.Size([33, 18])
torch.Size([44, 22])
torch.Size([43, 22])
torch.Size([28, 14])
torch.Size([45, 23])
torch.Size([18, 9])
torch.Size([38, 20])
torch.Size([45, 23])
torch.Size([32, 16])
torch.Size([22, 11])
torch.Size([31, 16])
torch.Size([36, 19])
torch.Size([45, 23])
torch.Size([33, 17])
torch.Size([28, 14])
torch.Size([26, 13])
torch.Size([46, 24])
torch.Size([43, 22])
torch.Size([28, 14])
torch.Size([26, 13])
torch.Size([32, 16])
torch.Size([38, 19])
torch.Size([42, 21])
torch.Size([30, 15])
torch.Size([30, 15])
torch.Size([38, 19])
torch.Size([32, 16])
torch.Size([28, 14])
torch.Size([36, 18])
torch.Size([29, 15])
torch.Size([50, 26])
torch.Size([65, 34])
torch.Size([26, 13])
torch.Size([40, 20])
torch.Size([21, 12])
torch.Size([23, 12])
torch.Size([38, 19])
torch.Size([38, 19])
torch.Size([22, 11])


In [35]:
label_pred_list

[0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,


In [36]:
y_true_list

[0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,


## Resultados

In [37]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [38]:
mean_precision = []
mean_recall = []
mean_f1 = []

prec = precision_score(y_true_list, label_pred_list, average='binary', labels=[0])
rec = recall_score(y_true_list, label_pred_list, average='binary', labels=[0])
f1 = f1_score(y_true_list, label_pred_list, average='binary', labels=[0])

In [39]:
print(classification_report(y_true_list,label_pred_list))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96       364
           1       0.99      0.92      0.95       339

    accuracy                           0.96       703
   macro avg       0.96      0.96      0.96       703
weighted avg       0.96      0.96      0.96       703



In [40]:
prec

0.9936102236421726

In [41]:
rec

0.9174041297935103

In [42]:
f1

0.9539877300613496